In [ ]:
%pip install -U langchain-openai

## Setting up LLM

In [44]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Create a GPT-4o model
llm = ChatOpenAI(model='gpt-4o',temperature=0,api_key=os.getenv("OPENAI_API_KEY"))

## Tools for LLM

In [54]:
import subprocess
from datetime import datetime
from langchain.tools import tool
from langchain.tools import StructuredTool

@tool
def github_documentation_commit(commit_message=""):
    """ Commits the current changes to the 'documentation' branch of the Github repository """
    branch_name = "documentation"
    commit_message_main = f"{commit_message} - {datetime.now().strftime('%d-%m-%y %H:%M:%S')}"
    
    # Switchs to the "documentation" branch (or Creates it if it doesn't exist)
    subprocess.run(["git", "checkout", "-B", branch_name])
    
    # Add all changes
    subprocess.run(["git", "add", "."])
    
    # Commit with the message
    subprocess.run(["git", "commit", "-m", commit_message_main])
    
    # Push to the remote repository
    subprocess.run(["git", "push", "-u", "origin", branch_name])
    
    return "Committed to Github on branch 'documentation'"

def create_file_tool(file_contents: str, file_path: str) -> str:
    """
    Function that creates/updates a markdown file with the contents provided as input 
    and file path provided as inputs
    """
    
    # Remove any Markdown formatting if present
    if file_contents.startswith("```") and file_contents.endswith("```"):
        file_contents = file_contents.strip("```").strip()
    
    with open(file_path, "w") as f:
        f.write(file_contents)
    
    return "File updated"

create_file_tool = StructuredTool.from_function(create_file_tool)
tools = [github_documentation_commit, create_file_tool]

## Simple Agent to use the tools

In [55]:
from langchain.agents import initialize_agent
from langchain.agents import AgentType

agent_executor = initialize_agent(
    tools, llm, agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [ ]:
agent_executor.invoke(
    {
        "input": "Write a basic code documentation in markdown format with the file name as documentation.md. Output ONLY THE MARKDOWN without wrapping it in ```. Commit changes to branch called documentation with an appropriate commit message."
    }
)



> Entering new AgentExecutor chain...
